# 基于LLM微调的数学推理任务

In [1]:
import os 
from tqdm import tqdm
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader
from torch.nn.functional import one_hot
from sklearn.metrics import accuracy_score, f1_score
from evaluate import load
from datasets import load_dataset

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

2024-12-07 17:00:22.043479: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-07 17:00:23.579296: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# # 设置代理
# import subprocess
# import os

# result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
# output = result.stdout
# for line in output.splitlines():
#     if '=' in line:
#         var, value = line.split('=', 1)
#         os.environ[var] = value

In [3]:
from datasets import load_dataset


test_dataset = load_dataset('json', data_files='./MATH_test.jsonl',split='train')
train_dataset = load_dataset('json', data_files='./MATH_train.jsonl', split='train')



In [4]:
def get_dataloader(prompt, batch_size=1):
    def preprocess(dataset, shuffle=True):
        def collate_fn(batch):
            questions =  [[{"role": "user", "content": prompt.format(text=item['problem'])}] for item in batch]
            answers = [item["solution"] for item in batch]
            return questions, answers
        return DataLoader(
            dataset,
            shuffle=shuffle,
            batch_size=batch_size,
            collate_fn=collate_fn,
        )
    testset = test_dataset
    return preprocess(testset, shuffle=False)

In [22]:
 ## setup math_equivalence ： 在math工作目录中 pip install .
import math_equivalence
from modelscope import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import json

def eval_process(model_name, tokenizer_name, prompt, batch_size=1, ratio=1):
    # 检查是否有可用的 GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # 加载模型和分词器到 GPU
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",         # 如果有多个 GPU，可以自动分配
        cache_dir='/autodl-tcachemp/'
    ).to(device)  # 确保模型在 GPU 上

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 加载数据
    testloader = get_dataloader(prompt, batch_size)
    print(testloader)
    
    answers = []
    replies = []

    # 遍历数据集
    for idx, (texts, truths) in tqdm(enumerate(testloader), total=int(len(testloader) * ratio)):
        if idx >= int(len(testloader) * ratio):
            break

        # 应用模板并生成输入
        texts = [tokenizer.apply_chat_template(
            text,
            tokenize=False,
            add_generation_prompt=True
        ) for text in texts]

        # 将输入张量移到 GPU 上
        model_inputs = tokenizer(
            texts,
            return_tensors="pt",
            padding=True,
            padding_side='left'
        ).to(device)

        # 在 GPU 上生成文本
        with torch.no_grad():  # 禁用梯度计算，节省显存
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=256
            )

        # 截取生成的结果
        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        # 解码生成的结果
        responses = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        replies.extend(responses)
        answers.extend(truths)
        print(len(replies))
    # 计算正确率
    # total = 0
    # correct = 0
    # for reply, answer in zip(replies, answers):
    #     total += 1
    #     if math_equivalence.is_equiv(reply, answer) :
    #         correct += 1
    # print(f"正确率: {(correct / total) * 100: .2f}%")
    results = [{"answer": a, "reply": r} for a, r in zip(answers, replies)]
    output_file = "results.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    print(f"Results saved to {output_file}")

    return replies,answers


In [23]:
model_name = "output/qwen2_5-0_5b-instruct/v1-20241207-153823/checkpoint-464"
tokenizer_name = "Qwen/Qwen2.5-0.5B-Instruct"

prompt = '''You are a highly skilled mathematician capable of solving complex grade-school math problems step by step. Please read the problem carefully and solve it with clear reasoning. Follow these instructions:

- Identify the known information and the question being asked.
- Break the solution into logical steps, providing clear explanations for each.
- Show all calculations and intermediate results.
- Conclude with a final answer.

Output your response in the following format:
[Explanation and calculations]
#### [Final numerical answer]   

Here is the problem:
{text}'''
replies,answers = eval_process(model_name, tokenizer_name, prompt, batch_size=32, ratio=1)

Using device: cuda


  1%|          | 1/157 [00:15<40:16, 15.49s/it]

32


  1%|▏         | 2/157 [00:34<44:49, 17.35s/it]

64


...

4960


 99%|█████████▉| 156/157 [42:41<00:14, 14.89s/it]

4992


100%|██████████| 157/157 [42:52<00:00, 16.39s/it]

5000
Results saved to results.json


In [13]:
# 导入swift框架进行微调
from swift.llm import (
    DatasetName, InferArguments, ModelType, SftArguments,
    infer_main, sft_main, app_ui_main
)

In [15]:
model_type = ModelType.qwen2_5_0_5b_instruct
sft_args = SftArguments(
    model_type=model_type,
    dataset=['MATH_train_fineture.jsonl'],
    output_dir='output',
    max_length=4096,
    sft_type = 'full',
    
    )
result = sft_main(sft_args)
last_model_checkpoint = result['last_model_checkpoint']

[INFO:swift] Setting template_type: qwen2_5
[INFO:swift] Setting args.lazy_tokenize: False
[INFO:swift] Setting args.dataloader_num_workers: 1
[INFO:swift] output_dir: /mnt/workspace/output/qwen2_5-0_5b-instruct/v1-20241207-153823
[INFO:swift] Start time of running main: 2024-12-07 15:38:23.736837
[INFO:swift] args: SftArguments(model_type='qwen2_5-0_5b-instruct', model_id_or_path='qwen/Qwen2.5-0.5B-Instruct', model_revision='master', full_determinism=False, sft_type='full', freeze_parameters=[], freeze_vit=False, freeze_parameters_ratio=0.0, additional_trainable_parameters=[], tuner_backend='peft', template_type='qwen2_5', output_dir='/mnt/workspace/output/qwen2_5-0_5b-instruct/v1-20241207-153823', add_output_dir_suffix=True, ddp_backend=None, ddp_find_unused_parameters=None, ddp_broadcast_buffers=None, ddp_timeout=1800, seed=42, resume_from_checkpoint=None, resume_only_model=False, ignore_data_skip=False, dtype='bf16', packing=False, train_backend='transformers', tp=1, pp=1, min_lr=N

device_count: 1
rank: -1, local_rank: -1, world_size: 1, local_world_size: 1


[WARNING:modelscope] Using branch: master as version is unstable, use with caution
[INFO:modelscope] Creating symbolic link /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2.5-0.5B-Instruct.
[WARNING:modelscope] Failed to create symbolic link /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2___5-0___5B-Instruct -> /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2.5-0.5B-Instruct: [Errno 17] File exists: '/mnt/workspace/.cache/modelscope/hub/qwen/Qwen2___5-0___5B-Instruct' -> '/mnt/workspace/.cache/modelscope/hub/qwen/Qwen2.5-0.5B-Instruct'
[INFO:swift] Loading the model using model_dir: /mnt/workspace/.cache/modelscope/hub/qwen/Qwen2___5-0___5B-Instruct
[INFO:swift] model_kwargs: {'device_map': 'cuda:0'}
[INFO:swift] model.max_model_len: 32768
[INFO:swift] model.hf_device_map: {'': device(type='cuda', index=0)}
[INFO:swift] model_config: Qwen2Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "/mnt/worksp

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

...

Val:   0%|          | 0/75 [00:00<?, ?it/s]

{'eval_loss': 0.69569194, 'eval_acc': 0.81994704, 'eval_runtime': 9.9632, 'eval_samples_per_second': 7.528, 'eval_steps_per_second': 7.528, 'epoch': 1.0, 'global_step/max_steps': '464/464', 'percentage': '100.00%', 'elapsed_time': '55m 43s', 'remaining_time': '0s'}


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
[INFO:swift] Saving model checkpoint to /mnt/workspace/output/qwen2_5-0_5b-instruct/v1-20241207-153823/checkpoint-464
[INFO:swift] last_model_checkpoint: /mnt/workspace/output/qwen2_5-0_5b-instruct/v1-20241207-153823/checkpoint-464
[INFO:swift] best_model_checkpoint: /mnt/workspace/output/qwen2_5-0_5b-instruct/v1-20241207-153823/checkpoint-464
[INFO:swift] images_dir: /mnt/workspace/output/qwen2_5-0_5b-instruct/v1-20241207-153823/images


{'train_runtime': 3348.4523, 'train_samples_per_second': 2.217, 'train_steps_per_second': 0.139, 'train_loss': 0.72626422, 'epoch': 1.0, 'global_step/max_steps': '464/464', 'percentage': '100.00%', 'elapsed_time': '55m 48s', 'remaining_time': '0s'}


[INFO:swift] End time of running main: 2024-12-07 16:34:40.772942
